In [ ]:
from datetime import datetime
import xarray as xr
import netCDF4
import matplotlib.pyplot as plt
%matplotlib inline
import cartopy.crs as ccrs
# import earthpy.plot as ep
import metpy  
import numpy as np
import s3fs
import glob

In [ ]:
year, month, day = 2018, 2, 7
doy = datetime(year, month, day).timetuple().tm_yday

In [ ]:
hour, minute = 12, 0

In [ ]:
# Use the anonymous credentials to access public data
fs = s3fs.S3FileSystem(anon=True)

# List contents of GOES-16 bucket.
files = fs.ls(f's3://noaa-goes16/ABI-L1b-RadF/{year}/{doy:03d}/{hour:02d}/')

files2down = []
for file in files:
    for c in ["C01", "C02", "C03"]:
        if c in str(file):
            files2down.append(file)
            print(file)
            fs.get(file, file.split('/')[-1])

In [ ]:
# Calculate latitude and longitude from GOES ABI fixed grid projection data
# GOES ABI fixed grid projection is a map projection relative to the GOES satellite
# Units: latitude in °N (°S < 0), longitude in °E (°W < 0)
# See GOES-R Product User Guide (PUG) Volume 5 (L2 products) Section 4.2.8 for details & example of calculations
# "file_id" is an ABI L1b or L2 .nc file opened using the netCDF4 library

def calculate_degrees(file_id):
    
    # Read in GOES ABI fixed grid projection variables and constants
    x_coordinate_1d = file_id.variables['x'][:]  # E/W scanning angle in radians
    y_coordinate_1d = file_id.variables['y'][:]  # N/S elevation angle in radians
    projection_info = file_id.variables['goes_imager_projection']
    lon_origin = projection_info.longitude_of_projection_origin
    H = projection_info.perspective_point_height+projection_info.semi_major_axis
    r_eq = projection_info.semi_major_axis
    r_pol = projection_info.semi_minor_axis
    
    # Create 2D coordinate matrices from 1D coordinate vectors
    x_coordinate_2d, y_coordinate_2d = np.meshgrid(x_coordinate_1d, y_coordinate_1d)
    
    # Equations to calculate latitude and longitude
    lambda_0 = (lon_origin*np.pi)/180.0  
    a_var = np.power(np.sin(x_coordinate_2d),2.0) + (np.power(np.cos(x_coordinate_2d),2.0)*(np.power(np.cos(y_coordinate_2d),2.0)+(((r_eq*r_eq)/(r_pol*r_pol))*np.power(np.sin(y_coordinate_2d),2.0))))
    b_var = -2.0*H*np.cos(x_coordinate_2d)*np.cos(y_coordinate_2d)
    c_var = (H**2.0)-(r_eq**2.0)
    r_s = (-1.0*b_var - np.sqrt((b_var**2)-(4.0*a_var*c_var)))/(2.0*a_var)
    s_x = r_s*np.cos(x_coordinate_2d)*np.cos(y_coordinate_2d)
    s_y = - r_s*np.sin(x_coordinate_2d)
    s_z = r_s*np.cos(x_coordinate_2d)*np.sin(y_coordinate_2d)
    
    # Ignore numpy errors for sqrt of negative number; occurs for GOES-16 ABI CONUS sector data
    np.seterr(all='ignore')
    
    abi_lat = (180.0/np.pi)*(np.arctan(((r_eq*r_eq)/(r_pol*r_pol))*((s_z/np.sqrt(((H-s_x)*(H-s_x))+(s_y*s_y))))))
    abi_lon = (lambda_0 - np.arctan(s_y/(H-s_x)))*(180.0/np.pi)
    
    return abi_lat, abi_lon

In [ ]:
c01 = xr.open_dataset(glob.glob(f'OR*-M3C01_G16_s{year:04d}{doy:03d}{hour:02d}{minute:02d}*.nc')[0])
c02 = xr.open_dataset(glob.glob(f'OR*-M3C02_G16_s{year:04d}{doy:03d}{hour:02d}{minute:02d}*.nc')[0])
c03 = xr.open_dataset(glob.glob(f'OR*-M3C03_G16_s{year:04d}{doy:03d}{hour:02d}{minute:02d}*.nc')[0])

In [ ]:
nc_c01 = netCDF4.Dataset(glob.glob(f'OR*-M3C01_G16_s{year:04d}{doy:03d}{hour:02d}{minute:02d}*.nc')[0])
abi_lat, abi_lon = calculate_degrees(nc_c01)

In [ ]:
print(abi_lon[2000, ::100])

In [ ]:
extent = [-34.0, 33.0, -22.0, 45.0]

In [ ]:
abi_lat_min = abi_lat.min(axis=1)
abi_lat_max = abi_lat.max(axis=1)
print(abi_lat_max[1128])
print(abi_lat_min[1128])
print(abi_lat_min[2537])
print(abi_lat_max[2537])


In [ ]:
print(abi_lon[1128,8000])

In [ ]:
abi_lon_min = abi_lon.min(axis=0)
abi_lon_max = abi_lon.max(axis=0)
print(abi_lon_min[8000])
print(abi_lon_max[8000])
print(abi_lon_min[-1])
print(abi_lon_max[-1])

In [ ]:
# Calculate the lat lon pairs indexes for the desired extent
idx_pair_1 = abs(abi_lat-extent[1])+abs(abi_lon-extent[0])
max_lat_idx,min_lon_idx = np.unravel_index(idx_pair_1.argmin(),idx_pair_1.shape)
idx_pair_2 = abs(abi_lat-extent[3])+abs(abi_lon-extent[2])
min_lat_idx,max_lon_idx = np.unravel_index(idx_pair_2.argmin(),idx_pair_2.shape)

In [ ]:
print(min_lat_idx, max_lat_idx, min_lon_idx, max_lon_idx)

In [ ]:
# print(abi_lon[min_lat_idx, min_lon_idx])
print(abi_lon[min_lat_idx, max_lon_idx])
print(abi_lon[max_lat_idx, min_lon_idx])
# print(abi_lon[max_lat_idx, max_lon_idx])
# print(abi_lat[min_lat_idx, min_lon_idx])
print(abi_lat[min_lat_idx, max_lon_idx])
print(abi_lat[max_lat_idx, min_lon_idx])
# print(abi_lat[max_lat_idx, max_lon_idx])

In [ ]:
ymin = extent[0].min()
ymax = extent[0].max()
xmin = extent[1].min()
xmax = extent[1].max()

In [ ]:
print(abi_lat[ymin, xmin],
abi_lat[ymin, xmax],
abi_lat[ymax, xmin],
abi_lat[ymax, xmax])

In [ ]:
print(abi_lon[ymin, xmin],
abi_lon[ymin, xmax],
abi_lon[ymax, xmin],
abi_lon[ymax, xmax])

In [ ]:
R = c02['Rad'].data
G = c03['Rad'].data
B = c01['Rad'].data

In [ ]:
print(B.shape)

In [ ]:
dat = c01.metpy.parse_cf('Rad')
goes = dat.metpy.cartopy_crs
x = dat.x
y = dat.y

In [ ]:
fig = plt.figure(figsize=(15, 12))

pc = ccrs.PlateCarree()

ax = fig.add_subplot(1, 1, 1, projection=pc)
ax.set_extent([-33, -23, 34, 44], crs=pc)

# Add the RGB image to the figure. The data is in the same projection as the
# axis we just created.
ax.imshow(RGB, origin='upper', extent=(x.min(), x.max(), y.min(), y.max()), transform=goes)

# Add Coastlines and States
# ax.coastlines(resolution='50m', color='green', linewidth=0.25)
# ax.add_feature(ccrs.cartopy.feature.STATES, linewidth=0.25, color='green')

plt.title('GOES-16 True Color', loc='left', fontweight='bold', fontsize=15)

plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 12))

# Create axis with Geostationary projection
ax = fig.add_subplot(1, 1, 1, projection=goes)

# Add the RGB image to the figure. The data is in the same projection as the
# axis we just created.
ax.imshow(B[ymin:ymax,xmin:xmax], origin='upper', cmap='Greys_r',
          extent=(y[ymin], y[ymax], x[xmin], x[xmax]), transform=goes)

# Add Coastlines and States
ax.coastlines(resolution='50m', color='black', linewidth=0.25)
ax.add_feature(ccrs.cartopy.feature.STATES, linewidth=0.25)

plt.title('GOES-16 True Color', loc='left', fontweight='bold', fontsize=15)

plt.show()

In [ ]:
# Plot the Blue Band radiances to check if data loaded correctly
fig = plt.figure(figsize=(6,6),dpi=200)
im = plt.imshow(B[ymin:ymax,xmin:xmax], cmap='Greys_r')
cb = fig.colorbar(im, orientation='horizontal')
cb.set_ticks([1, 100, 200, 300, 400, 500, 600])
cb.set_label('Radiance (W m-2 sr-1 um-1)')
plt.show()

In [ ]:
# Print the kappa coefficients for each band
kappa_B = c01['kappa0'].data
kappa_R = c02['kappa0'].data
kappa_G = c03['kappa0'].data

print('Band 1 kappa coefficient = ', kappa_B)
print('Band 2 kappa coefficient = ', kappa_R)
print('Band 3 kappa coefficient = ', kappa_G)

In [ ]:
#To convert radiance to reflectance, use formula:
#reflectance (ρf(υ)) = kappa factor(κ) * radiance (L(ν))
#Source: GOES-R Series Product Definition and User Guide (PUG) Volume 3, Revision 2.2, pages 27-28
R_ref = kappa_R * R
G_ref = kappa_G * G  
B_ref = kappa_B * B 

In [ ]:
# Apply range limits for each channel. Reflectance values must be between 0 and 1.
R_ref = np.clip(R_ref, 0, 1)
G_ref = np.clip(G_ref, 0, 1)
B_ref = np.clip(B_ref, 0, 1)

In [ ]:
# Apply a gamma correction to the image to correct ABI detector brightness
gamma = 2.2
R = np.power(R_ref, 1/gamma)
G = np.power(G_ref, 1/gamma)
B = np.power(B_ref, 1/gamma)

In [ ]:
print(R.shape)
print(G.shape)
print(B.shape)

In [ ]:
# Define the rebin function that will be used to resample the band resolution
# Rebin function from https://stackoverflow.com/questions/8090229/resize-with-averaging-or-rebin-a-numpy-2d-array
def rebin(a, shape):
    sh = shape[0],a.shape[0]//shape[0],shape[1],a.shape[1]//shape[1]
    return a.reshape(sh).mean(-1).mean(1)

In [ ]:
#Resample the Red Band resolution
R_rescaled = rebin(R, G.shape) 

In [ ]:
print(R_rescaled.shape)
print(G.shape)
print(B.shape)

In [ ]:
# GOES-R Series satellites do not have a channel in the visible green range. Band 3 is a NIR channel typically used to monitor vegetation.
# Calculate the "True" Green Band to serve as a green proxy for the RGB True Color image, using a fractional combination.
# Source: "Generation of GOES‐16 True Color Imagery without a Green Band" - https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2018EA000379
G_true = 0.45 * R_rescaled + 0.1 * G + 0.45 * B
G_true = np.clip(G_true, 0, 1)  # Apply band limits again, just in case.

In [ ]:
# The RGB array for the true color image
RGB = np.dstack([R_rescaled, G_true, B])

In [ ]:
fig = plt.figure(figsize=(15, 15))
ax = fig.add_axes((0.05, 0.05, 0.9, 0.9))

# True Color: RGB for the true color image
ax.imshow(RGB)
ax.set_title('GOES-16 RGB True Color', fontweight='bold', loc='left', fontsize=12)
ax.set_title(f'{year}{month:02d}{day:02d}{hour:02d}{minute:02d}', loc='right')
ax.axis('off')